<a href="https://colab.research.google.com/github/megancoll/megan_INFO5731_-Spring2022/blob/main/In_class_exercise/megan_in_class_exercise_05_11172022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The fifth in-class-exercise (40 points in total, 11/17/2022)**

(20 points) The purpose of the question is to practice different machine learning algorithms for text classification as well as the performance evaluation. In addition, you are requried to conduct *10 fold cross validation (https://scikit-learn.org/stable/modules/cross_validation.html)* in the training. 

The dataset can be download from canvas. The dataset contains two files train data and test data for sentiment analysis in IMDB review, it has two categories: 1 represents positive and 0 represents negative. You need to split the training data into training and validate data (80% for training and 20% for validation, https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6) and perform 10 fold cross validation while training the classifier. The final trained model was final evaluated on the test data. 

Algorithms:

(1) MultinominalNB

(2) SVM 

(3) KNN 

(4) Decision tree

(5) Random Forest

(6) XGBoost

Evaluation measurement:

(1) Accuracy

(2) Recall

(3) Precison 

(4) F-1 score

In [ ]:
# imports
import numpy as np
from google.colab import files
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import cross_validate
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report, silhouette_score
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
#upload zip file and unzip

uploaded = files.upload()

!unzip "/content/exercise05_datacollection.zip"


Saving exercise05_datacollection.zip to exercise05_datacollection.zip
Archive:  /content/exercise05_datacollection.zip
   creating: exercise09_datacollection/
  inflating: exercise09_datacollection/stsa-test.txt  
  inflating: exercise09_datacollection/stsa-train.txt  


In [ ]:
#opening the training and testing files

with open("/content/exercise09_datacollection/stsa-test.txt") as file:
    lines1 = [line.rstrip() for line in file]

with open("/content/exercise09_datacollection/stsa-train.txt") as file:
    lines2 = [line.rstrip() for line in file]

In [ ]:
#splitting labels from the text of the reviews

x_train_text = [l[2:] for l in lines1]

y_train = [int(l[0]) for l in lines1]


x_test_text = [l[2:] for l in lines2]

y_test = [int(l[0]) for l in lines2]

For this assignment, I will use bag of words as features. In a bag of words model, each word type in the training data is a possible feature. In this case, there are 7050 word types in the training data, so 7050 features.

In [ ]:
#tokenizing the texts into lists of words

x_train_tokenized = [word_tokenize(l) for l in x_train_text]
x_test_tokenized = [word_tokenize(l) for l in x_test_text]



In [ ]:
#this is to get a list of all the words in the training set

all_words = []
for l in x_train_tokenized:
  for word in l:
    all_words.append(word)

In [ ]:
all_words = list(set(all_words))

len(all_words)


7050

In [ ]:
#turning the training/testing data into lists of features (bag-of-words features)

X_train = []
X_test = []
for l in x_train_tokenized:
  x_train = []
  for word in all_words:
    if word in l:
      x_train.append(1)
    else:
      x_train.append(0)
  x_train = np.array(x_train)
  X_train.append(x_train)


for l in x_test_tokenized:
  x_test = []
  for word in all_words:
    if word in l:
      x_test.append(1)
    else:
      x_test.append(0)
  x_test = np.array(x_test)
  X_test.append(x_test)

Naive Bayes

In [ ]:
#naive bayes model
model_NB = MultinomialNB()
scoring = ['accuracy', 'precision_macro', 'recall_macro']
scores = cross_validate(model_NB, X_train, y_train, scoring=scoring, cv=10, return_estimator=True)

print(scores['test_accuracy'])

#choosing best model
max_acc = np.argmax(scores['test_accuracy'])
best_model = scores['estimator'][max_acc]

[0.73224044 0.77472527 0.7032967  0.75274725 0.7032967  0.8021978
 0.68131868 0.76923077 0.73626374 0.74175824]


In [ ]:
#evaluating chosen model on test data

y_pred = best_model.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.72      0.75      0.74      3310
           1       0.76      0.73      0.75      3610

    accuracy                           0.74      6920
   macro avg       0.74      0.74      0.74      6920
weighted avg       0.74      0.74      0.74      6920



In [ ]:
#turning the above code into function for reuse with different models

def run_experiment(model, mod_type):
  scoring = ['accuracy', 'precision_macro', 'recall_macro']
  scores = cross_validate(model, X_train, y_train, scoring=scoring, cv=10, return_estimator=True)

  print(f"{mod_type}: dev accuracy from cross-validation")
  print(scores['test_accuracy'])
  print("------------------------------------------------------------------")

  #choosing best model
  max_acc = np.argmax(scores['test_accuracy'])
  best_model = scores['estimator'][max_acc]

  #evaluating chosen model on test data
  y_pred = best_model.predict(X_test)
  print("Test Results:")
  print(classification_report(y_test, y_pred))

SVM

In [ ]:
model_SVM = svm.LinearSVC()

run_experiment(model_SVM, "SVM")


SVM: dev accuracy from cross-validation
[0.68306011 0.78571429 0.68131868 0.68681319 0.66483516 0.70879121
 0.66483516 0.77472527 0.76923077 0.67032967]
------------------------------------------------------------------
Test Results:
              precision    recall  f1-score   support

           0       0.69      0.73      0.71      3310
           1       0.74      0.71      0.72      3610

    accuracy                           0.72      6920
   macro avg       0.72      0.72      0.72      6920
weighted avg       0.72      0.72      0.72      6920



KNN

In [ ]:
model_KNN = neighbors.KNeighborsClassifier()

run_experiment(model_KNN, "KNN")

KNN: dev accuracy from cross-validation
[0.49726776 0.55494505 0.53846154 0.56593407 0.57692308 0.5989011
 0.48351648 0.57692308 0.60989011 0.58241758]
------------------------------------------------------------------
Test Results:
              precision    recall  f1-score   support

           0       0.51      0.75      0.61      3310
           1       0.61      0.35      0.44      3610

    accuracy                           0.54      6920
   macro avg       0.56      0.55      0.53      6920
weighted avg       0.56      0.54      0.52      6920



Decision Tree

In [ ]:
model_DecTree = tree.DecisionTreeClassifier()

run_experiment(model_DecTree, "Decision Tree")

Decision Tree: dev accuracy from cross-validation
[0.58469945 0.65384615 0.61538462 0.56043956 0.5989011  0.5989011
 0.63186813 0.64835165 0.65934066 0.56043956]
------------------------------------------------------------------
Test Results:
              precision    recall  f1-score   support

           0       0.59      0.58      0.59      3310
           1       0.62      0.63      0.63      3610

    accuracy                           0.61      6920
   macro avg       0.61      0.61      0.61      6920
weighted avg       0.61      0.61      0.61      6920



Random Forest

In [ ]:
model_rf = RandomForestClassifier()

run_experiment(model_rf, "Random Forest")

Random Forest: dev accuracy from cross-validation
[0.64480874 0.68131868 0.65384615 0.71978022 0.64835165 0.68681319
 0.64285714 0.69230769 0.72527473 0.69230769]
------------------------------------------------------------------
Test Results:
              precision    recall  f1-score   support

           0       0.65      0.67      0.66      3310
           1       0.69      0.66      0.68      3610

    accuracy                           0.67      6920
   macro avg       0.67      0.67      0.67      6920
weighted avg       0.67      0.67      0.67      6920



XGBoost

In [ ]:
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)

scoring = ['accuracy', 'precision_macro', 'recall_macro']

X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

scores = cross_validate(xgb_model, X_train, y_train, scoring=scoring, cv=10, return_estimator=True)

print("xgb_boost: dev accuracy from cross-validation")
print(scores['test_accuracy'])
print("------------------------------------------------------------------")

#choosing best model
max_acc = np.argmax(scores['test_accuracy'])
best_model = scores['estimator'][max_acc]

#evaluating chosen model on test data
y_pred = best_model.predict(X_test)
print("Test Results:")
print(classification_report(y_test, y_pred))

xgb_boost: dev accuracy from cross-validation
[0.62295082 0.65934066 0.61538462 0.63186813 0.64285714 0.69230769
 0.6043956  0.63736264 0.7032967  0.65934066]
------------------------------------------------------------------
Test Results:
              precision    recall  f1-score   support

           0       0.63      0.61      0.62      3310
           1       0.65      0.67      0.66      3610

    accuracy                           0.64      6920
   macro avg       0.64      0.64      0.64      6920
weighted avg       0.64      0.64      0.64      6920



Clustering

(20 points) The purpose of the question is to practice different machine learning algorithms for text clustering
Please downlad the dataset by using the following link.  https://www.kaggle.com/PromptCloudHQ/amazon-reviews-unlocked-mobile-phones
(You can also use different text data which you want)

Apply the listed clustering methods to the dataset:

K means, 
DBSCAN,
Hierarchical clustering. 

You can refer to of the codes from  the follwing link below. 
https://www.kaggle.com/karthik3890/text-clustering 

In [ ]:
#cleaning functions from https://www.kaggle.com/karthik3890/text-clustering
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer which is developed in recent years
stop=set(stopwords.words('english'))



def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('************************************')
print(sno.stem('tasty'))

{'whom', 'was', 'm', "won't", 'ourselves', 'be', "mustn't", 'being', 're', 'couldn', 'and', "she's", "you'll", 'there', 'your', 'about', 'of', 'then', 'ain', 'is', 'has', 'does', 'below', 'yourselves', 'more', 'isn', 'very', 'under', 'during', 'o', 'll', "couldn't", 'after', 'should', 'now', 'too', 'in', 'hadn', 'my', 'on', 'only', "aren't", 'hers', 'for', 'most', 'weren', 'against', 'd', "doesn't", 'me', 'ours', "mightn't", 'what', 'will', 'to', 'haven', 'doesn', 'from', 'wouldn', 'such', 'once', 'where', 'shouldn', 'am', 'been', 'its', "wasn't", 'same', 'but', 'how', 'can', 'our', 'up', 'that', 'by', 'had', 'until', "you've", 'doing', 'before', 'she', 'theirs', 'shan', 'who', 'no', 'into', 's', 'with', 'off', 'have', "hasn't", 'i', 'needn', "should've", 'it', 'an', 'each', "didn't", "hadn't", 'further', 'are', 'at', 'nor', 'if', 'any', "you'd", 't', "shan't", 'out', 'hasn', 'not', 'mustn', 'here', 'won', 'him', "that'll", 'all', 'itself', 'do', 'this', "needn't", 've', 'over', 'might

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!unzip "/content/archive.zip"

Archive:  /content/archive.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/archive.zip or
        /content/archive.zip.zip, and cannot find /content/archive.zip.ZIP, period.


In [ ]:
!unzip "/content/Amazon_Unlocked_Mobile.csv.zip"

Archive:  /content/Amazon_Unlocked_Mobile.csv.zip
  inflating: Amazon_Unlocked_Mobile.csv  


In [ ]:
m_df = pd.read_csv("/content/Amazon_Unlocked_Mobile.csv")

In [ ]:
m_df

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0
...,...,...,...,...,...,...
413835,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,another great deal great price,0.0
413836,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,Ok,0.0
413837,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,5,Passes every drop test onto porcelain tile!,0.0
413838,Samsung Convoy U640 Phone for Verizon Wireless...,Samsung,79.95,3,I returned it because it did not meet my needs...,0.0


In [ ]:
#cleaning functions from https://www.kaggle.com/karthik3890/text-clustering
i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in m_df['Reviews'].values:
    filtered_sentence=[]
    #print(sent);
    sent = str(sent)
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                else:
                    continue
            else:
                continue 
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1

In [ ]:
m_df['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 
m_df['CleanedText']=m_df['CleanedText'].str.decode("utf-8")

In [ ]:
#Write your code here.
sents = list(m_df.loc[:, "CleanedText"])

sents = [str(s) for s in sents]

vectorizer = CountVectorizer()
vects = vectorizer.fit_transform(sents)

In [ ]:
vects.shape

(413840, 46648)

Fitting the clustering algorithms

In [ ]:
#k means
from sklearn.cluster import KMeans

model = KMeans(n_clusters = 10,init='k-means++',random_state=99)
model.fit(vects)

KMeans(n_clusters=10, random_state=99)

In [ ]:
#DBSCAN
from sklearn.cluster import DBSCAN

model = DBSCAN()

model.fit(vects)

DBSCAN()

In [ ]:
#heirarchical
#this one will not run because my system does not have enough RAM so my code breaks every time after this
from sklearn.cluster import AgglomerativeClustering


model = AgglomerativeClustering()
vects = vects.toarray()
model.fit(vects)

NameError: ignored

In one paragraph, please compare K means, DBSCAN and Hierarchical clustering.

K means is a partitional clustering algorithm, while DBSCAN is a density based clustering algorithm, and Heirarchical clustering is an agglomerative based clustering method. K means is useful because it is fast and efficient, but the desired number of clusters must be specified beforehand. In K means, points are assigned to the cluster for which they are closest to the centroid. As points are reassigned, the centroids are recalculated, until the clusters stabilize. K means is not as good at handling outliers or clusters with strange shapes. DBSCAN is good for handling outliers and strange shaped clusters, and the number of clusters does not need to be specified beforehand. Clusters are calculated by grouping points together which all have a certain number of neighbors within a specified radius. Hierarchical Clustering uses linking distance to merge things into the same cluster. 